In [1]:
import os
os.chdir("..")
!pwd
!pyenv version
%load_ext autoreload
%autoreload 2

/Users/loicmorel/code/projetcs/crypto_assistant
crypto_assistant (set by PYENV_VERSION environment variable)


In [3]:
import pandas as pd
import numpy as np
from app.data_mgt.datamgt import ohlcv_from_csv_to_df
from tqdm import trange
from scipy import stats
from sklearn.linear_model import LinearRegression

In [4]:
file = 'data/ohlcv_BTCUSDT_1h_from_1502942400000_to_1651507200000.csv'
data = ohlcv_from_csv_to_df(file)
data

,o_ts,Open,High,Low,Close,Volume,c_ts,o_date
0,2017-08-17 04:00:00,4261.48,4313.62,4261.32,4308.83,47.181009,2017-08-17 04:59:59.999000064,17395.166667
1,2017-08-17 05:00:00,4308.83,4328.69,4291.37,4315.32,23.234916,2017-08-17 05:59:59.999000064,17395.208333
2,2017-08-17 06:00:00,4330.29,4345.45,4309.37,4324.35,7.229691,2017-08-17 06:59:59.999000064,17395.250000
3,2017-08-17 07:00:00,4316.62,4349.99,4287.41,4349.99,4.443249,2017-08-17 07:59:59.999000064,17395.291667
4,2017-08-17 08:00:00,4333.32,4377.85,4333.32,4360.69,0.972807,2017-08-17 08:59:59.999000064,17395.333333
...,...,...,...,...,...,...,...,...
41138,2022-05-02 12:00:00,38528.78,38705.00,38422.92,38500.41,1530.249700,2022-05-02 12:59:59.999000064,19114.500000
41139,2022-05-02 13:00:00,38500.42,39155.47,38242.51,38910.00,7532.509850,2022-05-02 13:59:59.999000064,19114.541667
41140,2022-05-02 14:00:00,38916.27,39104.02,38606.21,38840.00,6534.617490,2022-05-02 14:59:59.999000064,19114.583333
41141,2022-05-02 15:00:00,38840.00,38901.37,38605.71,38696.05,2520.982230,2022-05-02 15:59:59.999000064,19114.625000


In [5]:
from app.indicators import find_swings
data_ = find_swings(data)

In [6]:
from app.indicators import classify_swings
data__ = classify_swings(data_)

In [7]:
data__

,Highs,Lows,Last,Trend,CSLS,Support,Resistance,Strong_lows,Strong_highs,FSL,FSH,LL,HL,HH,LH
0,4313.62,4261.32,4313.62,0.0,0,False,False,0.00,0.00,0.00,0.00,False,False,False,False
1,4328.69,4291.37,4328.69,NaN,0,False,False,0.00,0.00,0.00,0.00,False,False,False,False
2,0.00,4261.32,4261.32,1.0,0,False,False,0.00,0.00,0.00,0.00,False,False,False,False
3,0.00,4261.32,4261.32,1.0,1,True,False,4261.32,0.00,4261.32,0.00,False,True,False,False
4,0.00,4261.32,4261.32,1.0,2,True,False,4261.32,0.00,0.00,0.00,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41138,39075.00,38782.29,39075.00,-1.0,2,False,True,0.00,39075.00,0.00,0.00,False,False,False,False
41139,39075.00,38422.92,38422.92,1.0,0,False,False,0.00,0.00,0.00,0.00,False,False,False,False
41140,39075.00,38422.92,38422.92,1.0,1,True,False,38422.92,0.00,38422.92,0.00,False,True,False,False
41141,39155.47,38422.92,39155.47,-1.0,0,False,False,0.00,0.00,0.00,0.00,False,False,False,False


In [8]:
data__=data__*1

In [9]:
data__

,Highs,Lows,Last,Trend,CSLS,Support,Resistance,Strong_lows,Strong_highs,FSL,FSH,LL,HL,HH,LH
0,4313.62,4261.32,4313.62,0.0,0,0,0,0.00,0.00,0.00,0.00,0,0,0,0
1,4328.69,4291.37,4328.69,NaN,0,0,0,0.00,0.00,0.00,0.00,0,0,0,0
2,0.00,4261.32,4261.32,1.0,0,0,0,0.00,0.00,0.00,0.00,0,0,0,0
3,0.00,4261.32,4261.32,1.0,1,1,0,4261.32,0.00,4261.32,0.00,0,1,0,0
4,0.00,4261.32,4261.32,1.0,2,1,0,4261.32,0.00,0.00,0.00,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41138,39075.00,38782.29,39075.00,-1.0,2,0,1,0.00,39075.00,0.00,0.00,0,0,0,0
41139,39075.00,38422.92,38422.92,1.0,0,0,0,0.00,0.00,0.00,0.00,0,0,0,0
41140,39075.00,38422.92,38422.92,1.0,1,1,0,38422.92,0.00,38422.92,0.00,0,1,0,0
41141,39155.47,38422.92,39155.47,-1.0,0,0,0,0.00,0.00,0.00,0.00,0,0,0,0


In [10]:
df=pd.merge(data,data__,left_index=True,right_index=True)

In [11]:
df=df[['o_ts','Open','High','Low','Close','Volume','LL','HL','HH','LH']]

In [12]:
df

,o_ts,Open,High,Low,Close,Volume,LL,HL,HH,LH
0,2017-08-17 04:00:00,4261.48,4313.62,4261.32,4308.83,47.181009,0,0,0,0
1,2017-08-17 05:00:00,4308.83,4328.69,4291.37,4315.32,23.234916,0,0,0,0
2,2017-08-17 06:00:00,4330.29,4345.45,4309.37,4324.35,7.229691,0,0,0,0
3,2017-08-17 07:00:00,4316.62,4349.99,4287.41,4349.99,4.443249,0,1,0,0
4,2017-08-17 08:00:00,4333.32,4377.85,4333.32,4360.69,0.972807,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
41138,2022-05-02 12:00:00,38528.78,38705.00,38422.92,38500.41,1530.249700,0,0,0,0
41139,2022-05-02 13:00:00,38500.42,39155.47,38242.51,38910.00,7532.509850,0,0,0,0
41140,2022-05-02 14:00:00,38916.27,39104.02,38606.21,38840.00,6534.617490,0,1,0,0
41141,2022-05-02 15:00:00,38840.00,38901.37,38605.71,38696.05,2520.982230,0,0,0,0


In [13]:
df['sum'] = df[['HH','HL','LL','LH']].sum(axis=1)
df['NP']=df['sum'].apply(lambda x:1 if x==0 else 0)
df.drop(columns='sum',inplace=True)

In [13]:
df

,o_ts,Open,High,Low,Close,Volume,LL,HL,HH,LH,NP
0,2017-08-17 04:00:00,4261.48,4313.62,4261.32,4308.83,47.181009,0,0,0,0,1
1,2017-08-17 05:00:00,4308.83,4328.69,4291.37,4315.32,23.234916,0,0,0,0,1
2,2017-08-17 06:00:00,4330.29,4345.45,4309.37,4324.35,7.229691,0,0,0,0,1
3,2017-08-17 07:00:00,4316.62,4349.99,4287.41,4349.99,4.443249,0,1,0,0,0
4,2017-08-17 08:00:00,4333.32,4377.85,4333.32,4360.69,0.972807,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
41138,2022-05-02 12:00:00,38528.78,38705.00,38422.92,38500.41,1530.249700,0,0,0,0,1
41139,2022-05-02 13:00:00,38500.42,39155.47,38242.51,38910.00,7532.509850,0,0,0,0,1
41140,2022-05-02 14:00:00,38916.27,39104.02,38606.21,38840.00,6534.617490,0,1,0,0,0
41141,2022-05-02 15:00:00,38840.00,38901.37,38605.71,38696.05,2520.982230,0,0,0,0,1


In [14]:
df.set_index('o_ts',inplace=True)

In [16]:
df.to_csv('data/transformed_data/BTCUSDT_1h_transformed.csv')

In [17]:
df[['HH','HL','LL','LH','NP']].value_counts()

HH  HL  LL  LH  NP
0   0   0   0   1     31203
    1   0   0   0      3043
    0   0   1   0      2672
1   0   0   0   0      2133
0   0   1   0   0      2092
dtype: int64

In [18]:
def subsample_sequence(df, length):
    """
    Given the initial dataframe `df`, return a shorter dataframe sequence of length `length`.
    This shorter sequence should be selected at random
    """
    last_possible = df.shape[0] - length
    
    random_start = np.random.randint(0, last_possible)
    df_sample = df[random_start: random_start+length]
    return df_sample

In [19]:
assert subsample_sequence(df, 10).shape == (10, 10)
assert subsample_sequence(df, 10000).shape == (10000, 10)

In [20]:
def split_subsample_sequence(df, length):
    '''Create one single random (X,y) pair'''
    df_subsample = subsample_sequence(df, length)
    y_sample = df_subsample.iloc[length -1][['LL','HL','HH','LH','NP']]
    
    X_sample = df_subsample[0:length -1]
    X_sample = X_sample.values
    return np.array(X_sample), np.array(y_sample)

In [21]:
(X_sample, y_sample) = split_subsample_sequence(df, 10)
assert X_sample.shape == (9,10)
assert y_sample.shape == (5,)

In [22]:
def get_X_y(df, n_sequences, length):
    '''Return a list of samples (X, y)'''
    X, y = [], []

    for i in range(n_sequences):
        (xi, yi) = split_subsample_sequence(df, length)
        X.append(xi)
        y.append(yi)
        
    X = np.array(X)
    y = np.array(y)
    return X, y

In [23]:
X, y = get_X_y(df, 2000, 21)
print(X.shape)
print(y.shape)

(2000, 20, 10)
(2000, 5)


In [24]:
assert np.isnan(X).sum() == 0

In [25]:
len_ = int(0.8*df.shape[0])
data_train = df[:len_]
data_test = df[len_:]

In [26]:
X_train, y_train = get_X_y(data_train, 2000, 21)
X_test, y_test = get_X_y(data_test, 400, 21)

In [27]:
assert X_train.shape == (2000, 20, 10)

In [28]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers, metrics
from tensorflow.keras.layers.experimental.preprocessing import Normalization

In [29]:


def init_model():
    opt = optimizers.RMSprop(learning_rate=0.01)
    
    normalizer = Normalization()
    normalizer.adapt(X_train)

    model = models.Sequential()
    model.add(normalizer)
    model.add(layers.GRU(units=20, activation='tanh', return_sequences=True))
    model.add(layers.GRU(units=20, activation='tanh', return_sequences=False))
    model.add(layers.Dense(50, activation='relu'))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(5, activation='softmax'))

    
    model.compile(loss='categorical_crossentropy', 
                  optimizer=opt, 
                  metrics=['accuracy'])
    
    return model

In [30]:
model = init_model()
model.summary()

2022-05-08 22:38:38.965517: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-08 22:38:39.022362: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
normalization (Normalization (None, None, 10)          21        
_________________________________________________________________
gru (GRU)                    (None, None, 20)          1920      
_________________________________________________________________
gru_1 (GRU)                  (None, 20)                2520      
_________________________________________________________________
dense (Dense)                (None, 50)                1050      
_________________________________________________________________
dropout (Dropout)            (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 255       
Total params: 5,766
Trainable params: 5,745
Non-trainable params: 21
_____________________________________________________

In [31]:
from tensorflow.keras.callbacks import EarlyStopping

es=EarlyStopping(monitor='accuracy', patience=10)

In [32]:
class_weight = {0: 5.,
                1: 5.,
                2: 5.,
                3: 5.,
                4: 1.}

In [33]:
model.fit(X_train, y_train, 
          epochs=75, 
          batch_size=64, 
          verbose=1, 
          callbacks = [es],
          validation_split=0.2,class_weight=class_weight)

Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
Epoch 1/75
25/25 [==============================] - 2s 25ms/step - loss: 2.7964 - accuracy: 0.6000 - val_loss: 0.8918 - val_accuracy: 0.7475
Epoch 2/75
25/25 [==============================] - 0s 8ms/step - loss: 2.4209 - accuracy: 0.5412 - val_loss: 0.9071 - val_accuracy: 0.4825
Epoch 3/75
25/25 [==============================] - 0s 8ms/step - loss: 2.2999 - accuracy: 0.5006 - val_loss: 0.8609 - val_accuracy: 0.5475
Epoch 4/75
25/25 [==============================] - 0s 8ms/step - loss: 2.2523 - accuracy: 0.5056 - val_loss: 0.8548 - val_accuracy: 0.5500
Epoch 5/75
25/25 [==============================] - 0s 8ms/step - loss: 2.2049 - accuracy: 0.5106 - val_loss: 0.9278 - val_accuracy: 0.4400
Epoch 6/75
25/25 [==============================] - 0s 8ms/step - loss: 2.1586 - accuracy: 0.5206 - val_loss: 0.9337 - val_accuracy: 0.5175
Epoch 7/75
25/25 [===

In [34]:
model.evaluate(X_test, y_test)

13/13 [==============================] - 0s 2ms/step - loss: 1.0081 - accuracy: 0.4775


[1.0080801248550415, 0.47749999165534973]

In [35]:
np.round(model.predict(X_test),2)

array([[0.  , 0.01, 0.56, 0.01, 0.42],
       [0.24, 0.22, 0.  , 0.01, 0.53],
       [0.16, 0.28, 0.07, 0.24, 0.25],
       ...,
       [0.03, 0.59, 0.  , 0.  , 0.38],
       [0.  , 0.04, 0.14, 0.34, 0.47],
       [0.  , 0.  , 0.  , 0.  , 1.  ]], dtype=float32)

model.save('./models/RNN_BTCUSDT_1h.h5')

# Prediction based on input

In [36]:
df_AAVE=pd.read_csv('data/transformed_data/AAVEUSDT_1h_transformed.csv')
df_BTC=pd.read_csv('data/transformed_data/BTCUSDT_1h_transformed.csv')
df_DOT=pd.read_csv('data/transformed_data/DOTUSDT_1h_transformed.csv')
df_ETH=pd.read_csv('data/transformed_data/ETHUSDT_1h_transformed.csv')
df_FTM=pd.read_csv('data/transformed_data/FTMUSDT_1h_transformed.csv')
df_NEAR=pd.read_csv('data/transformed_data/NEARUSDT_1h_transformed.csv')
df_SOL=pd.read_csv('data/transformed_data/SOLUSDT_1h_transformed.csv')
df_VITE=pd.read_csv('data/transformed_data/VITEUSDT_1h_transformed.csv')

params={'AAVE':df_AAVE,'BTC':df_BTC,'DOT':df_DOT,'ETH':df_ETH,'FTM':df_FTM,'NEAR':df_NEAR,'SOL':df_SOL,'VITE':df_VITE}
labels={0:'LL',1:'HL',2:'HH',3:'LH',4:'NP'}

FileNotFoundError: [Errno 2] No such file or directory: 'data/transformed_data/AAVEUSDT_1h_transformed.csv'

In [43]:
def model_rnn_prediction(name, date):
            df=params.get(name)
            index=df.index
            row_num=index[df['o_ts']==date]
            index_slices=range(row_num[0]-20,row_num[0])
            X=df.loc[index_slices]
            X=X.drop(columns='o_ts')
            X_3d=np.array([X])
            
            # model=models.load_model(f'./models/RNN_{name}USDT_1h.h5')
            pred=model.predict_classes(X_3d)
            
            return labels.get(pred[0])

In [45]:
model_rnn_prediction('BTC', '2017-08-18 03:00:00.000000000')

/Users/heather/.pyenv/versions/3.8.12/envs/crypto_assistant/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


'HL'